In [3]:
#!/usr/bin/env python
import os

In [91]:
# set up
mainDirectory = "/Users/carlrothfels/Desktop/PURC_analysesForPaper/6_runsWchimeraCountsOutput/"
outDirectory = "/Users/carlrothfels/Box Sync/R_Python_etal/git_repositories/ppp_repo/writing_the_paper/"
outHandle = "chimeraCounts.csv"

loci = ["APP", "GAP", "IBR", "PGI"]
regimes = ["a", "c", "e"]
stringentRegimes = ["Str_" + regime for regime in regimes]
regimes.extend(stringentRegimes)

rounds = ["R2"] #3, 4 could be included
os.chdir(mainDirectory)

#os.getcwd()

In [92]:
# Counting the chimeras found
chimeraCounts = {}
for thisround in rounds:
    for regime in regimes:
        print "REGIME ", regime
        lineCount = 0
        chimeraSection = False
        os.chdir(mainDirectory + thisround + regime)
        countsFile = open("purc_cluster_counts.xls", 'rU')
        
        for line in countsFile:
            if "**Chimeric clusters" in line: #ie, we've got to the section of the file with the counts of chimeras
                chimeraSection = True
            if chimeraSection:
                lineCount += 1
                if lineCount > 1: # So that we skip the "**Chimeric clusters ..." line (the header)
                    line = line.strip("\n")
                    line = line.split("\t")
                    if len(line[0]) > 0: # If there's something in the first column, it is the locus name
                        locus = line[0]
                        #print "Working on ", locus
                        chimeraCounts[thisround + regime, locus] = [0,0,0,0] #Make an entry in the dictionary with a key like "[R2a, GAP]"
                        #print chimeraCounts
                    else:
                        for i in range(2,6):
                            if len(line) >2: #The blank entries have a line of length two: the first tab, and then the taxon name
                                # The [i-2] business is because I've set i to go through the columns I want (3,4,5,6), but need to index the list by (1,2,3,4)
                                chimeraCounts[thisround + regime, locus][i-2] += int(line[i])                  
        countsFile.close()
            

REGIME  a
REGIME  c
REGIME  e
REGIME  Str_a
REGIME  Str_c
REGIME  Str_e


In [107]:
# Printing it all to a CSV file
os.chdir(outDirectory)
outfile = open(outHandle, 'w')
for thisround in rounds:
    for regime in regimes: 
        for locus in loci:
            outfile.write(thisround + regime + ",") # printing the regime name (1st column)
            outfile.write(locus + ",") # printing the locus name
            #print chimeraCounts[thisround + regime, locus]
            for count in chimeraCounts[thisround + regime, locus]:
                outfile.write(str(count) + ",")
            outfile.write("\n")
outfile.close()